In [1]:
import requests
import pandas as pd
import json
import base64
import os
from datetime import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import traceback
from dotenv import load_dotenv

# Cargar variables de entorno desde el archivo .env
load_dotenv()

# Configuración de VALD
CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')
# FECHA_DESDE = os.getenv('FECHA_DESDE')
FECHA_DESDE= '2023-01-01T00:00:00.000Z'

# Crear directorio para guardar los CSV si no existe
OUTPUT_DIR = "output_data"
os.makedirs(OUTPUT_DIR, exist_ok=True)



In [14]:
# Función para obtener token
def get_token():
    token_url = "https://security.valdperformance.com/connect/token"
    
    payload = {
        "grant_type": "client_credentials",
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET
    }
    
    try:
        response = requests.post(token_url, data=payload)
        
        if response.status_code == 200:
            token_data = response.json()
            print("✅ Autenticación exitosa")
            return token_data.get('access_token')
        else:
            print(f"❌ Error en la autenticación: {response.status_code}")
            print(response.text)
            return None
            
    except Exception as e:
        print(f"❌ Error inesperado: {str(e)}")
        return None

# Función para obtener tenants
def get_tenants(token):
    url = "https://prd-use-api-externaltenants.valdperformance.com/tenants"
    
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/json"
    }
    
    try:
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            tenants_data = response.json()
            df_tenants = pd.DataFrame(tenants_data['tenants'])
            
            # Guardar a CSV
            csv_path = os.path.join(OUTPUT_DIR, "tenants.csv")
            df_tenants.to_csv(csv_path, index=False)
            print(f"✅ Datos de tenants guardados en {csv_path}")
            
            return df_tenants
        else:
            print(f"❌ Error al obtener tenants: {response.status_code}")
            return pd.DataFrame()
            
    except Exception as e:
        print(f"❌ Error inesperado: {str(e)}")
        return pd.DataFrame()

# Función para obtener categorías
def get_categories(tenant_id, token):
    url = "https://prd-use-api-externaltenants.valdperformance.com/categories"
    
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/json"
    }
    
    params = {
        "TenantId": tenant_id
    }
    
    try:
        print("🔄 Solicitando categories a la API...")
        response_categories = requests.get(url, headers=headers, params=params)
        
        if response_categories.status_code != 200:
            print(f"❌ Error al obtener categorías: {response_categories.status_code}")
            return pd.DataFrame()
            
        categories = json.loads(response_categories.content.decode('utf-8'))
        df_categories_ = pd.DataFrame(categories['categories'])
        df_categories_['tenant_id'] = tenant_id
        
        # Guardar a CSV
        csv_path = os.path.join(OUTPUT_DIR, f"categories_{tenant_id}.csv")
        df_categories_.to_csv(csv_path, index=False)
        print(f"✅ Categorías para tenant {tenant_id} guardadas en {csv_path}")
        
        return df_categories_
        
    except Exception as e:
        print(f"❌ Error al obtener categorías: {str(e)}")
        return pd.DataFrame()

# Función para obtener grupos
def get_groups(tenant_id, token):
    url = "https://prd-use-api-externaltenants.valdperformance.com/groups"
    
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/json"
    }
    
    params = {
        "TenantId": tenant_id
    }
    
    try:
        print("🔄 Solicitando grupos a la API...")
        response_groups = requests.get(url, headers=headers, params=params)
        
        if response_groups.status_code != 200:
            print(f"❌ Error al obtener grupos: {response_groups.status_code}")
            return pd.DataFrame()
            
        groups = json.loads(response_groups.content.decode('utf-8'))
        df_groups_ = pd.DataFrame(groups['groups'])
        df_groups_['tenant_id'] = tenant_id
        
        # Guardar a CSV
        csv_path = os.path.join(OUTPUT_DIR, f"groups_{tenant_id}.csv")
        df_groups_.to_csv(csv_path, index=False)
        print(f"✅ Grupos para tenant {tenant_id} guardados en {csv_path}")
        
        return df_groups_
        
    except Exception as e:
        print(f"❌ Error al obtener grupos: {str(e)}")
        return pd.DataFrame()

# Función para obtener perfiles
def get_profiles(token, tenant_id, groupId, groupName, categoryId, categoryName, df_all_profiles=None):
    url = "https://prd-use-api-externalprofile.valdperformance.com/profiles"
    
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/json"
    }
    
    params = {
        "TenantId": tenant_id,
        "groupId": groupId
    }
    
    try:
        print(f"🔄 Solicitando perfiles para grupo {groupName}...")
        response_profiles = requests.get(url, headers=headers, params=params)
        
        if response_profiles.status_code != 200:
            print(f"⚠️ Error HTTP {response_profiles.status_code} para grupo {groupName}")
            return df_all_profiles if df_all_profiles is not None else pd.DataFrame()
        
        if not response_profiles.content or response_profiles.content == b'':
            print(f"⚠️ Respuesta vacía para grupo {groupName}")
            return df_all_profiles if df_all_profiles is not None else pd.DataFrame()
        
        content_str = response_profiles.content.decode('utf-8')
        
        if not content_str.strip():
            print(f"⚠️ Contenido vacío para grupo {groupName}")
            return df_all_profiles if df_all_profiles is not None else pd.DataFrame()
            
        profiles = json.loads(content_str)
        
        if 'profiles' not in profiles:
            print(f"⚠️ Estructura JSON inesperada para grupo {groupName}")
            return df_all_profiles if df_all_profiles is not None else pd.DataFrame()
        
        if not profiles['profiles']:
            print(f"ℹ️ No hay perfiles para el grupo {groupName}")
            return df_all_profiles if df_all_profiles is not None else pd.DataFrame()
        
        df_profiles_ = pd.DataFrame(profiles['profiles'])
        
        df_profiles_['tenant_id'] = tenant_id
        df_profiles_['groupId'] = groupId
        df_profiles_['groupName'] = groupName
        df_profiles_['categoryId'] = categoryId
        df_profiles_['categoryName'] = categoryName
        
        # Si ya tenemos un DataFrame de todos los perfiles, concatenamos
        if df_all_profiles is not None:
            df_all_profiles = pd.concat([df_all_profiles, df_profiles_], ignore_index=True)
        else:
            df_all_profiles = df_profiles_
        
        print(f"✅ {len(df_profiles_)} perfiles obtenidos para grupo {groupName}")
        
        return df_all_profiles
        
    except json.JSONDecodeError as e:
        print(f"❌ Error JSON para grupo {groupName}: {e}")
        return df_all_profiles if df_all_profiles is not None else pd.DataFrame()
    except Exception as e:
        print(f"❌ Error inesperado para grupo {groupName}: {e}")
        return df_all_profiles if df_all_profiles is not None else pd.DataFrame()


def get_nordbord_complete(token, tenant_id, fecha_desde, profile_id=None):
    """
    Obtiene TODOS los datos de NordBord usando la paginación correcta del endpoint /tests/v2
    """
    base_url = "https://prd-use-api-externalnordbord.valdperformance.com"
    endpoint = "/tests/v2"
    
    headers = {
        "Authorization": f"Bearer {token}"
    }
    
    all_tests = []  # Cambiado para ser más específico
    current_modified_from = fecha_desde
    page_count = 0
    
    print(f"🔄 Iniciando obtención completa de datos NordBord para tenant {tenant_id}...")
    
    while True:
        page_count += 1
        print(f"📄 Procesando página {page_count} (desde: {current_modified_from})")
        
        params = {
            "tenantId": tenant_id,
            "modifiedFromUtc": current_modified_from
        }
        
        if profile_id:
            params["profileId"] = profile_id
        
        try:
            response = requests.get(f"{base_url}{endpoint}", params=params, headers=headers)
            
            if response.status_code == 200:
                datos = response.json()
                
                # Determinar la estructura de los datos
                if isinstance(datos, list):
                    current_batch = datos
                elif isinstance(datos, dict):
                    # Buscar diferentes posibles keys
                    if 'tests' in datos:
                        current_batch = datos['tests']
                    elif 'items' in datos:
                        current_batch = datos['items']
                    elif 'data' in datos:
                        current_batch = datos['data']
                    else:
                        # Si es un dict sin keys conocidas, tratarlo como un solo elemento
                        current_batch = [datos]
                else:
                    current_batch = [datos] if datos else []
                
                if not current_batch:
                    print("✅ No hay más datos en esta respuesta")
                    break
                
                print(f"📊 Obtenidos {len(current_batch)} registros en esta página")
                
                # Agregar datos a la lista principal
                all_tests.extend(current_batch)
                
                # CLAVE: Buscar el campo de fecha de modificación para paginación
                last_record = current_batch[-1]
                next_modified_date = None
                
                # Buscar diferentes posibles nombres para el campo de fecha
                possible_date_fields = ['modifiedDateUtc', 'modifiedDate', 'lastModified', 
                                      'updatedAt', 'dateModified', 'modified']
                
                for field in possible_date_fields:
                    if field in last_record:
                        next_modified_date = last_record[field]
                        print(f"🔄 Encontrado campo '{field}': {next_modified_date}")
                        break
                
                if next_modified_date:
                    # Verificar si la fecha es la misma que la anterior (bucle infinito)
                    if next_modified_date == current_modified_from:
                        print("⚠️ Detectado bucle infinito: misma fecha de modificación")
                        print("🔄 Agregando 1 milisegundo para avanzar...")
                        
                        # Convertir a datetime, agregar 1 milisegundo, y volver a string
                        from datetime import datetime, timedelta
                        try:
                            dt = datetime.fromisoformat(next_modified_date.replace('Z', '+00:00'))
                            dt_next = dt + timedelta(milliseconds=1)
                            current_modified_from = dt_next.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z'
                            print(f"🔄 Nueva fecha: {current_modified_from}")
                        except Exception as e:
                            print(f"❌ Error al procesar fecha: {e}")
                            break
                    else:
                        current_modified_from = next_modified_date
                        print(f"🔄 Siguiente página desde: {current_modified_from}")
                else:
                    print("⚠️ No se encontró campo de fecha de modificación en el último registro")
                    print(f"⚠️ Campos disponibles: {list(last_record.keys())}")
                    break
                
                # Protección adicional contra bucles infinitos
                if page_count > 1000:  # Máximo 1000 páginas
                    print("⚠️ Alcanzado límite máximo de páginas (1000). Deteniendo...")
                    break
                
            elif response.status_code == 204:
                print("✅ Código 204 - No hay más registros para obtener")
                break
                
            else:
                print(f"❌ Error {response.status_code}: {response.text}")
                break
                
        except Exception as e:
            print(f"❌ Error en la solicitud: {str(e)}")
            break
    
    # Convertir a DataFrame
    if all_tests:
        df = pd.DataFrame(all_tests)
        
        # Convertir columnas de fecha (sin el warning)
        date_columns = [col for col in df.columns 
                       if 'date' in col.lower() or 'time' in col.lower()]
        for col in date_columns:
            try:
                df[col] = pd.to_datetime(df[col])
            except:
                pass  # Si no se puede convertir, mantener el formato original
        
        # Añadir información del tenant
        df['tenant_id'] = tenant_id
        
        print(f"🎉 Proceso completado: {len(df)} registros totales obtenidos en {page_count} páginas")
        return df
    else:
        print("⚠️ No se obtuvieron datos")
        return pd.DataFrame()



In [15]:
# Obtener token
token = get_token()
if not token:
    print("❌ No se pudo obtener el token. Proceso cancelado.")

# Obtener tenants
df_tenants = get_tenants(token)
if df_tenants.empty:
    print("❌ No se pudieron obtener los tenants. Proceso cancelado.")

# Inicializar DataFrames para consolidar datos
df_all_profiles = pd.DataFrame()
df_all_nordbord = pd.DataFrame()

✅ Autenticación exitosa
✅ Datos de tenants guardados en output_data/tenants.csv


In [16]:
tenant_id = df_tenants['id'].loc[0]
tenant_name = df_tenants['name'].loc[0]
print(tenant_id)
print(tenant_name)

f1185650-fb79-44a0-8b4b-b2bf82d28c83
STAFF - Luciano Tomagheli


In [17]:
# Obtener categorías
df_categories = get_categories(tenant_id, token)
df_categories

🔄 Solicitando categories a la API...
✅ Categorías para tenant f1185650-fb79-44a0-8b4b-b2bf82d28c83 guardadas en output_data/categories_f1185650-fb79-44a0-8b4b-b2bf82d28c83.csv


,id,syncId,name,tenant_id
0,d9a94657-6111-4993-a5fe-081b21f15b75,None,Import,f1185650-fb79-44a0-8b4b-b2bf82d28c83
1,58734421-a5ed-420c-a9d5-26c5b0eda5d4,None,Categorias,f1185650-fb79-44a0-8b4b-b2bf82d28c83
2,f021a645-f179-49bd-9181-30a433c19672,None,Test,f1185650-fb79-44a0-8b4b-b2bf82d28c83
3,eac290e8-77a8-45ee-85cd-3c15d8acb9f0,None,CBMM Hockey,f1185650-fb79-44a0-8b4b-b2bf82d28c83
4,00031b91-0b46-445e-ab45-40fe3b4964c7,None,Vald CSM,f1185650-fb79-44a0-8b4b-b2bf82d28c83
5,a48fa3ba-e1da-4502-8ddb-4558fb7210ba,None,Agile,f1185650-fb79-44a0-8b4b-b2bf82d28c83
6,f6093938-5d89-44d4-b0fa-48c30423b0b6,None,fuerzas basicas,f1185650-fb79-44a0-8b4b-b2bf82d28c83
7,82f698f2-7a4b-41bb-958b-5f01defafd28,None,Deporte,f1185650-fb79-44a0-8b4b-b2bf82d28c83
8,83a00bde-14e5-4d67-8fd4-6011b6118fa8,None,Proyecto Crossfit,f1185650-fb79-44a0-8b4b-b2bf82d28c83
9,8a53265b-a484-4fbb-9ae0-64e1a0daf47f,None,HSI,f1185650-fb79-44a0-8b4b-b2bf82d28c83


In [18]:
df_categories = df_categories[df_categories['name'] == 'CBMM']
df_categories

,id,syncId,name,tenant_id
24,0e7bfdff-aabb-4efa-ad13-ecb419e0beef,None,CBMM,f1185650-fb79-44a0-8b4b-b2bf82d28c83


In [19]:
# Obtener grupos
df_groups = get_groups(tenant_id, token)
df_groups

🔄 Solicitando grupos a la API...
✅ Grupos para tenant f1185650-fb79-44a0-8b4b-b2bf82d28c83 guardados en output_data/groups_f1185650-fb79-44a0-8b4b-b2bf82d28c83.csv


,id,name,categoryId,syncId,tenant_id
0,c6987553-a658-461c-9fe2-d09879d2cb00,1,5ad747f7-41e7-44b4-a0c3-864efa8a8aa8,None,f1185650-fb79-44a0-8b4b-b2bf82d28c83
1,2d907f1f-caff-4e4b-9790-8bd304b0fbba,2,5ad747f7-41e7-44b4-a0c3-864efa8a8aa8,None,f1185650-fb79-44a0-8b4b-b2bf82d28c83
2,db806a4b-8ae2-40f7-a5a0-7061f3026589,3,5ad747f7-41e7-44b4-a0c3-864efa8a8aa8,None,f1185650-fb79-44a0-8b4b-b2bf82d28c83
3,84cfb5d3-cd9d-49b9-83c2-30748bc8524d,4,5ad747f7-41e7-44b4-a0c3-864efa8a8aa8,None,f1185650-fb79-44a0-8b4b-b2bf82d28c83
4,b5d30982-020a-4c63-ab6e-f60db009878e,4ta,58734421-a5ed-420c-a9d5-26c5b0eda5d4,None,f1185650-fb79-44a0-8b4b-b2bf82d28c83
...,...,...,...,...,...
106,04f8f2b4-39dd-4dec-8131-d04f20aea0c0,Velez 7ma,5bac6d7d-2999-48e3-90e9-b84f788ed31f,None,f1185650-fb79-44a0-8b4b-b2bf82d28c83
107,9f98fb01-0cbe-4ed6-9e3b-b78b37c91097,Velez 8va,5bac6d7d-2999-48e3-90e9-b84f788ed31f,None,f1185650-fb79-44a0-8b4b-b2bf82d28c83
108,228d7637-5e50-4491-adb0-745fbcedd0d8,Velez 9na,5bac6d7d-2999-48e3-90e9-b84f788ed31f,None,f1185650-fb79-44a0-8b4b-b2bf82d28c83
109,6d04667e-5dc9-4172-8281-5b7da1c669a7,Wing,abfd3e98-0024-491a-95e9-cd8d91a9d0e8,None,f1185650-fb79-44a0-8b4b-b2bf82d28c83


In [20]:
df_groups = df_groups[df_groups['categoryId'] == '0e7bfdff-aabb-4efa-ad13-ecb419e0beef']

In [21]:
# Si hay categorías y grupos, combinarlos
if not df_categories.empty and not df_groups.empty:
    # Combinar grupos con categorías
    df_groups_with_category = df_groups.merge(
        df_categories[['id', 'name']].rename(columns={'name': 'category_name'}),
        left_on='categoryId',
        right_on='id',
        how='left'
    )
    
    # Limpiar columnas duplicadas
    if 'id_y' in df_groups_with_category.columns:
        df_groups_with_category = df_groups_with_category.drop(columns=['id_y'])
        df_groups_with_category = df_groups_with_category.rename(columns={'id_x': 'id'})
    
    # Guardar grupos con categorías
    csv_path = os.path.join(OUTPUT_DIR, f"groups_with_categories_{tenant_id}.csv")
    df_groups_with_category.to_csv(csv_path, index=False)
    print(f"✅ Grupos con categorías guardados en {csv_path}")
    
    # Obtener perfiles por grupo
    print("\n📊 Obteniendo perfiles para cada grupo...")
    for idx, group in df_groups_with_category.iterrows():
        df_all_profiles = get_profiles(
            token, 
            tenant_id, 
            group['id'], 
            group['name'], 
            group['categoryId'], 
            group.get('category_name', ''),
            df_all_profiles
        )
    

✅ Grupos con categorías guardados en output_data/groups_with_categories_f1185650-fb79-44a0-8b4b-b2bf82d28c83.csv

📊 Obteniendo perfiles para cada grupo...
🔄 Solicitando perfiles para grupo Inactivos...
✅ 4 perfiles obtenidos para grupo Inactivos
🔄 Solicitando perfiles para grupo M15 2025...
✅ 42 perfiles obtenidos para grupo M15 2025
🔄 Solicitando perfiles para grupo M16 2025...
✅ 37 perfiles obtenidos para grupo M16 2025
🔄 Solicitando perfiles para grupo M17 2025...
✅ 52 perfiles obtenidos para grupo M17 2025
🔄 Solicitando perfiles para grupo M19 2025...
✅ 62 perfiles obtenidos para grupo M19 2025
🔄 Solicitando perfiles para grupo Plantel Superior...
✅ 169 perfiles obtenidos para grupo Plantel Superior


In [22]:
df_all_profiles

,profileId,syncId,givenName,familyName,dateOfBirth,externalId,beingMergedWithProfileId,beingMergedWithProfileExpiryDateUtc,tenant_id,groupId,groupName,categoryId,categoryName
0,1dd48a20-d55d-4be3-9ce1-1ef41ea4e3a5,None,IGNACIO,RUIZ,2007-02-02T00:00:00,,None,None,f1185650-fb79-44a0-8b4b-b2bf82d28c83,35a2a19b-f8ea-4963-92b4-98565cf1e030,Inactivos,0e7bfdff-aabb-4efa-ad13-ecb419e0beef,CBMM
1,63016c13-015e-483f-864e-77b8117ed937,None,VALENTINO,PRECIOZA,2007-10-04T00:00:00,None,None,None,f1185650-fb79-44a0-8b4b-b2bf82d28c83,35a2a19b-f8ea-4963-92b4-98565cf1e030,Inactivos,0e7bfdff-aabb-4efa-ad13-ecb419e0beef,CBMM
2,d9c83db9-7479-4610-959a-7ffbf7334f30,None,BAUTISTA,JAUREGUIBERRY,2007-01-14T00:00:00,,None,None,f1185650-fb79-44a0-8b4b-b2bf82d28c83,35a2a19b-f8ea-4963-92b4-98565cf1e030,Inactivos,0e7bfdff-aabb-4efa-ad13-ecb419e0beef,CBMM
3,1de99470-2457-4e8d-8891-c69b92fcdecf,None,FELIX,DE BATTISTA,2007-01-09T00:00:00,,None,None,f1185650-fb79-44a0-8b4b-b2bf82d28c83,35a2a19b-f8ea-4963-92b4-98565cf1e030,Inactivos,0e7bfdff-aabb-4efa-ad13-ecb419e0beef,CBMM
4,e4b9df41-8f92-4470-859d-02e2c0c82b6c,None,BENJAMIN,DOS SANTOS,2010-05-30T00:00:00,49626725,None,None,f1185650-fb79-44a0-8b4b-b2bf82d28c83,31edcc44-ec7e-4ed3-adeb-74e1e803228e,M15 2025,0e7bfdff-aabb-4efa-ad13-ecb419e0beef,CBMM
...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,961222de-6cc1-4cce-9d56-fa7b61589d0a,None,Facu,Marino,2005-01-09T00:00:00,None,None,None,f1185650-fb79-44a0-8b4b-b2bf82d28c83,a1ecef8c-d185-4e45-951c-b6540e390eef,Plantel Superior,0e7bfdff-aabb-4efa-ad13-ecb419e0beef,CBMM
362,c4987963-2287-489b-a769-fb003bda1560,None,Bravi,Antonio,1982-06-08T00:00:00,None,None,None,f1185650-fb79-44a0-8b4b-b2bf82d28c83,a1ecef8c-d185-4e45-951c-b6540e390eef,Plantel Superior,0e7bfdff-aabb-4efa-ad13-ecb419e0beef,CBMM
363,9f42d5d4-0e2b-485e-9e91-fb2215036f0f,None,Ignacio,Papa,2005-01-04T00:00:00,None,None,None,f1185650-fb79-44a0-8b4b-b2bf82d28c83,a1ecef8c-d185-4e45-951c-b6540e390eef,Plantel Superior,0e7bfdff-aabb-4efa-ad13-ecb419e0beef,CBMM
364,59200686-dd45-4f09-8d8a-fc5fad0417db,None,Matias,Robinson,2005-01-05T00:00:00,None,None,None,f1185650-fb79-44a0-8b4b-b2bf82d28c83,a1ecef8c-d185-4e45-951c-b6540e390eef,Plantel Superior,0e7bfdff-aabb-4efa-ad13-ecb419e0beef,CBMM


In [23]:
df_all_profiles[df_all_profiles['familyName']=='SALDAÑA']

,profileId,syncId,givenName,familyName,dateOfBirth,externalId,beingMergedWithProfileId,beingMergedWithProfileExpiryDateUtc,tenant_id,groupId,groupName,categoryId,categoryName
156,1d4ec7c4-68b1-49f5-92ae-620c89d0032a,None,PATRICIO,SALDAÑA,2007-02-03T00:00:00,None,None,None,f1185650-fb79-44a0-8b4b-b2bf82d28c83,69db9f0b-106b-4b5a-97b8-0a283a488e10,M19 2025,0e7bfdff-aabb-4efa-ad13-ecb419e0beef,CBMM


# Esta ejecucion al no tener paginacion solo me trae los 50 primeros

In [24]:
#token, tenant_id, FECHA_DESDE,

base_url = "https://prd-use-api-externalnordbord.valdperformance.com"
    
endpoint = "/tests/v2"
params = {
    "tenantId": tenant_id,
    "modifiedFromUtc": FECHA_DESDE
}

headers = {
    "Authorization": f"Bearer {token}"
}


print(f"🔄 Solicitando datos NordBord para tenant {tenant_id}...")
response = requests.get(f"{base_url}{endpoint}", params=params, headers=headers)

if response.status_code == 200:
    print(f"✅ Datos obtenidos correctamente")
    datos = response.json()

    if isinstance(datos, dict) and 'items' in datos:
        df = pd.DataFrame(datos['items'])
    elif isinstance(datos, list):
                df = pd.DataFrame(datos)
    else:
        df = pd.DataFrame([datos])
expanded_df = pd.json_normalize(df['tests'].iloc[0])
print(len(expanded_df))

🔄 Solicitando datos NordBord para tenant f1185650-fb79-44a0-8b4b-b2bf82d28c83...
✅ Datos obtenidos correctamente
50


# Esta ejecucion tiene paginacion 

### extrae desde el 01-01-2023 23 paginas trayendo 1032 test pero no encuentra mas registros despues de 12-05-2025 y eso no es verdad porque en plataforma el jugador SALDAÑA tiene mas test por ejemplo el dia 17-07-2025


📄 Procesando página 23 (desde: 2025-05-13T23:08:49.165Z)

✅ Código 204 - No hay más registros para obtener

🎉 Proceso completado: 1032 registros totales obtenidos en 23 páginas

In [33]:
import requests
import pandas as pd
from datetime import datetime

def get_forcedecks_complete(token, tenant_id, fecha_desde, profile_id=None):
    """
    Obtiene TODOS los datos de NordBord usando la paginación correcta del endpoint /tests/v2
    """
    base_url = "https://prd-use-api-extforcedecks.valdperformance.com"
    endpoint = "/tests"
    
    headers = {
        "Authorization": f"Bearer {token}"
    }
    
    all_tests = []  # Cambiado para ser más específico
    current_modified_from = fecha_desde
    page_count = 0
    
    print(f"🔄 Iniciando obtención completa de datos ForceDecks para tenant {tenant_id}...")
    
    while True:
        page_count += 1
        print(f"📄 Procesando página {page_count} (desde: {current_modified_from})")
        
        params = {
            "tenantId": tenant_id,
            "modifiedFromUtc": current_modified_from
        }
        
        if profile_id:
            params["profileId"] = profile_id
        
        try:
            response = requests.get(f"{base_url}{endpoint}", params=params, headers=headers)
            
            if response.status_code == 200:
                datos = response.json()
                
                # Determinar la estructura de los datos
                if isinstance(datos, list):
                    current_batch = datos
                elif isinstance(datos, dict):
                    # Buscar diferentes posibles keys
                    if 'tests' in datos:
                        current_batch = datos['tests']
                    elif 'items' in datos:
                        current_batch = datos['items']
                    elif 'data' in datos:
                        current_batch = datos['data']
                    else:
                        # Si es un dict sin keys conocidas, tratarlo como un solo elemento
                        current_batch = [datos]
                else:
                    current_batch = [datos] if datos else []
                
                if not current_batch:
                    print("✅ No hay más datos en esta respuesta")
                    break
                
                print(f"📊 Obtenidos {len(current_batch)} registros en esta página")
                
                # Agregar datos a la lista principal
                all_tests.extend(current_batch)
                
                # CLAVE: Buscar el campo de fecha de modificación para paginación
                last_record = current_batch[-1]
                next_modified_date = None
                
                # Buscar diferentes posibles nombres para el campo de fecha
                possible_date_fields = ['modifiedDateUtc', 'modifiedDate', 'lastModified', 
                                      'updatedAt', 'dateModified', 'modified']
                
                for field in possible_date_fields:
                    if field in last_record:
                        next_modified_date = last_record[field]
                        print(f"🔄 Encontrado campo '{field}': {next_modified_date}")
                        break
                
                if next_modified_date:
                    # Verificar si la fecha es la misma que la anterior (bucle infinito)
                    if next_modified_date == current_modified_from:
                        print("⚠️ Detectado bucle infinito: misma fecha de modificación")
                        print("🔄 Agregando 1 milisegundo para avanzar...")
                        
                        # Convertir a datetime, agregar 1 milisegundo, y volver a string
                        from datetime import datetime, timedelta
                        try:
                            dt = datetime.fromisoformat(next_modified_date.replace('Z', '+00:00'))
                            dt_next = dt + timedelta(milliseconds=1)
                            current_modified_from = dt_next.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z'
                            print(f"🔄 Nueva fecha: {current_modified_from}")
                        except Exception as e:
                            print(f"❌ Error al procesar fecha: {e}")
                            break
                    else:
                        current_modified_from = next_modified_date
                        print(f"🔄 Siguiente página desde: {current_modified_from}")
                else:
                    print("⚠️ No se encontró campo de fecha de modificación en el último registro")
                    print(f"⚠️ Campos disponibles: {list(last_record.keys())}")
                    break
                
                # Protección adicional contra bucles infinitos
                if page_count > 1000:  # Máximo 1000 páginas
                    print("⚠️ Alcanzado límite máximo de páginas (1000). Deteniendo...")
                    break
                
            elif response.status_code == 204:
                print("✅ Código 204 - No hay más registros para obtener")
                break
                
            else:
                print(f"❌ Error {response.status_code}: {response.text}")
                break
                
        except Exception as e:
            print(f"❌ Error en la solicitud: {str(e)}")
            break
    
    # Convertir a DataFrame
    if all_tests:
        df = pd.DataFrame(all_tests)
        
        # Convertir columnas de fecha (sin el warning)
        date_columns = [col for col in df.columns 
                       if 'date' in col.lower() or 'time' in col.lower()]
        for col in date_columns:
            try:
                df[col] = pd.to_datetime(df[col])
            except:
                pass  # Si no se puede convertir, mantener el formato original
        
        # Añadir información del tenant
        df['tenant_id'] = tenant_id
        
        print(f"🎉 Proceso completado: {len(df)} registros totales obtenidos en {page_count} páginas")
        return df
    else:
        print("⚠️ No se obtuvieron datos")
        return pd.DataFrame()



In [34]:
df_forcedecks = get_forcedecks_complete(token, tenant_id, FECHA_DESDE)

🔄 Iniciando obtención completa de datos ForceDecks para tenant f1185650-fb79-44a0-8b4b-b2bf82d28c83...
📄 Procesando página 1 (desde: 2023-01-01T00:00:00.000Z)
📊 Obtenidos 50 registros en esta página
🔄 Encontrado campo 'modifiedDateUtc': 2023-02-09T18:49:26.385Z
🔄 Siguiente página desde: 2023-02-09T18:49:26.385Z
📄 Procesando página 2 (desde: 2023-02-09T18:49:26.385Z)
📊 Obtenidos 51 registros en esta página
🔄 Encontrado campo 'modifiedDateUtc': 2023-02-14T18:13:46.201Z
🔄 Siguiente página desde: 2023-02-14T18:13:46.201Z
📄 Procesando página 3 (desde: 2023-02-14T18:13:46.201Z)
📊 Obtenidos 50 registros en esta página
🔄 Encontrado campo 'modifiedDateUtc': 2023-03-13T17:13:05.878Z
🔄 Siguiente página desde: 2023-03-13T17:13:05.878Z
📄 Procesando página 4 (desde: 2023-03-13T17:13:05.878Z)
📊 Obtenidos 50 registros en esta página
🔄 Encontrado campo 'modifiedDateUtc': 2023-03-13T23:12:35.636Z
🔄 Siguiente página desde: 2023-03-13T23:12:35.636Z
📄 Procesando página 5 (desde: 2023-03-13T23:12:35.636Z)
📊

/var/folders/k1/j4nrrlzs2t93tp860tb51m9m0000gn/T/ipykernel_18299/1692729950.py:130: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col])
/var/folders/k1/j4nrrlzs2t93tp860tb51m9m0000gn/T/ipykernel_18299/1692729950.py:130: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col])


In [35]:
len(df_forcedecks)

3635

In [36]:
def get_dynamo_complete(token, tenant_id, fecha_desde, profile_id=None):
    """
    Obtiene TODOS los datos de Dynamo usando la paginación correcta del endpoint /tests
    """
    base_url = "https://prd-use-api-extdynamo.valdperformance.com"
    endpoint = "/tests"
    
    headers = {
        "Authorization": f"Bearer {token}"
    }
    
    all_tests = []  # Cambiado para ser más específico
    current_modified_from = fecha_desde
    page_count = 0
    
    print(f"🔄 Iniciando obtención completa de datos Dynamo para tenant {tenant_id}...")
    
    while True:
        page_count += 1
        print(f"📄 Procesando página {page_count} (desde: {current_modified_from})")
        
        params = {
            "tenantId": tenant_id,
            "modifiedFromUtc": current_modified_from
        }
        
        if profile_id:
            params["profileId"] = profile_id
        
        try:
            response = requests.get(f"{base_url}{endpoint}", params=params, headers=headers)
            
            if response.status_code == 200:
                datos = response.json()
                
                # Determinar la estructura de los datos
                if isinstance(datos, list):
                    current_batch = datos
                elif isinstance(datos, dict):
                    # Buscar diferentes posibles keys
                    if 'tests' in datos:
                        current_batch = datos['tests']
                    elif 'items' in datos:
                        current_batch = datos['items']
                    elif 'data' in datos:
                        current_batch = datos['data']
                    else:
                        # Si es un dict sin keys conocidas, tratarlo como un solo elemento
                        current_batch = [datos]
                else:
                    current_batch = [datos] if datos else []
                
                if not current_batch:
                    print("✅ No hay más datos en esta respuesta")
                    break
                
                print(f"📊 Obtenidos {len(current_batch)} registros en esta página")
                
                # Agregar datos a la lista principal
                all_tests.extend(current_batch)
                
                # CLAVE: Buscar el campo de fecha de modificación para paginación
                last_record = current_batch[-1]
                next_modified_date = None
                
                # Buscar diferentes posibles nombres para el campo de fecha
                possible_date_fields = ['modifiedDateUtc', 'modifiedDate', 'lastModified', 
                                      'updatedAt', 'dateModified', 'modified']
                
                for field in possible_date_fields:
                    if field in last_record:
                        next_modified_date = last_record[field]
                        print(f"🔄 Encontrado campo '{field}': {next_modified_date}")
                        break
                
                if next_modified_date:
                    # Verificar si la fecha es la misma que la anterior (bucle infinito)
                    if next_modified_date == current_modified_from:
                        print("⚠️ Detectado bucle infinito: misma fecha de modificación")
                        print("🔄 Agregando 1 milisegundo para avanzar...")
                        
                        # Convertir a datetime, agregar 1 milisegundo, y volver a string
                        from datetime import datetime, timedelta
                        try:
                            dt = datetime.fromisoformat(next_modified_date.replace('Z', '+00:00'))
                            dt_next = dt + timedelta(milliseconds=1)
                            current_modified_from = dt_next.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z'
                            print(f"🔄 Nueva fecha: {current_modified_from}")
                        except Exception as e:
                            print(f"❌ Error al procesar fecha: {e}")
                            break
                    else:
                        current_modified_from = next_modified_date
                        print(f"🔄 Siguiente página desde: {current_modified_from}")
                else:
                    print("⚠️ No se encontró campo de fecha de modificación en el último registro")
                    print(f"⚠️ Campos disponibles: {list(last_record.keys())}")
                    break
                
                # Protección adicional contra bucles infinitos
                if page_count > 1000:  # Máximo 1000 páginas
                    print("⚠️ Alcanzado límite máximo de páginas (1000). Deteniendo...")
                    break
                
            elif response.status_code == 204:
                print("✅ Código 204 - No hay más registros para obtener")
                break
                
            else:
                print(f"❌ Error {response.status_code}: {response.text}")
                break
                
        except Exception as e:
            print(f"❌ Error en la solicitud: {str(e)}")
            break
    
    # Convertir a DataFrame
    if all_tests:
        df = pd.DataFrame(all_tests)
        
        # Convertir columnas de fecha (sin el warning)
        date_columns = [col for col in df.columns 
                       if 'date' in col.lower() or 'time' in col.lower()]
        for col in date_columns:
            try:
                df[col] = pd.to_datetime(df[col])
            except:
                pass  # Si no se puede convertir, mantener el formato original
        
        # Añadir información del tenant
        df['tenant_id'] = tenant_id
        
        print(f"🎉 Proceso completado: {len(df)} registros totales obtenidos en {page_count} páginas")
        return df
    else:
        print("⚠️ No se obtuvieron datos")
        return pd.DataFrame()

In [37]:
df_dynamo = get_dynamo_complete(token, tenant_id, FECHA_DESDE)

🔄 Iniciando obtención completa de datos Dynamo para tenant f1185650-fb79-44a0-8b4b-b2bf82d28c83...
📄 Procesando página 1 (desde: 2023-01-01T00:00:00.000Z)
❌ Error 404: 
⚠️ No se obtuvieron datos


In [27]:
df_nordbord = get_nordbord_complete(token, tenant_id, FECHA_DESDE)


🔄 Iniciando obtención completa de datos NordBord para tenant f1185650-fb79-44a0-8b4b-b2bf82d28c83...
📄 Procesando página 1 (desde: 2023-01-01T00:00:00.000Z)
📊 Obtenidos 50 registros en esta página
🔄 Encontrado campo 'modifiedDateUtc': 2023-02-10T23:36:42.354Z
🔄 Siguiente página desde: 2023-02-10T23:36:42.354Z
📄 Procesando página 2 (desde: 2023-02-10T23:36:42.354Z)
📊 Obtenidos 50 registros en esta página
🔄 Encontrado campo 'modifiedDateUtc': 2023-03-06T23:47:11.341Z
🔄 Siguiente página desde: 2023-03-06T23:47:11.341Z
📄 Procesando página 3 (desde: 2023-03-06T23:47:11.341Z)
📊 Obtenidos 50 registros en esta página
🔄 Encontrado campo 'modifiedDateUtc': 2023-03-28T23:34:44.151Z
🔄 Siguiente página desde: 2023-03-28T23:34:44.151Z
📄 Procesando página 4 (desde: 2023-03-28T23:34:44.151Z)
📊 Obtenidos 50 registros en esta página
🔄 Encontrado campo 'modifiedDateUtc': 2023-04-19T12:00:32.307Z
🔄 Siguiente página desde: 2023-04-19T12:00:32.307Z
📄 Procesando página 5 (desde: 2023-04-19T12:00:32.307Z)
📊 O

In [19]:
df_all_profiles[df_all_profiles['familyName']=='SALDAÑA']

,profileId,syncId,givenName,familyName,dateOfBirth,externalId,tenant_id,groupId,groupName,categoryId,categoryName
150,1d4ec7c4-68b1-49f5-92ae-620c89d0032a,None,PATRICIO,SALDAÑA,2007-02-03T00:00:00,,f1185650-fb79-44a0-8b4b-b2bf82d28c83,69db9f0b-106b-4b5a-97b8-0a283a488e10,M19 2025,0e7bfdff-aabb-4efa-ad13-ecb419e0beef,CBMM


In [20]:
df_all_profiles[df_all_profiles['profileId']=='1d4ec7c4-68b1-49f5-92ae-620c89d0032a']

,profileId,syncId,givenName,familyName,dateOfBirth,externalId,tenant_id,groupId,groupName,categoryId,categoryName
150,1d4ec7c4-68b1-49f5-92ae-620c89d0032a,None,PATRICIO,SALDAÑA,2007-02-03T00:00:00,,f1185650-fb79-44a0-8b4b-b2bf82d28c83,69db9f0b-106b-4b5a-97b8-0a283a488e10,M19 2025,0e7bfdff-aabb-4efa-ad13-ecb419e0beef,CBMM


In [29]:
df_nordbord[df_nordbord['profileId']=='c2200d7c-6c9b-469d-98e0-4321d6ba1af0']

,profileId,testId,modifiedDateUtc,testDateUtc,testTypeId,testTypeName,notes,device,leftAvgForce,leftImpulse,...,leftTorque,leftCalibration,leftRepetitions,rightAvgForce,rightImpulse,rightMaxForce,rightTorque,rightCalibration,rightRepetitions,tenant_id
1110,c2200d7c-6c9b-469d-98e0-4321d6ba1af0,db894d42-a938-40ef-8bfa-256a275f697c,2025-08-05 22:35:54.903000+00:00,2025-08-05 22:35:54.680000+00:00,f9755ba4-17cf-4c47-9c3d-a8995905073d,ISO Prone,,NordBord-1756,298.0,4076.89,...,111.583125,0.0,2,402.0,6336.41,486.0,181.521,0.0,3,f1185650-fb79-44a0-8b4b-b2bf82d28c83


In [30]:
len(df_nordbord)

1117

# OTRAS PRUEBAS

### despues probe con otros filtros de fechas pero siempre trae hasta el 13-05-2025

📈 Distribución por año:

   2023: 4 tests
   
   2024: 226 tests

   2025: 119 tests

📊 Registros obtenidos: 349

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def get_nordbord_complete_and_filter(token, tenant_id, test_from_date="2023-01-01", test_to_date=None):
    """
    SOLUCIÓN FINAL: Usa el endpoint que funciona y filtra después por fecha del test
    
    Args:
        token: Token de autorización
        tenant_id: ID del tenant
        test_from_date: Fecha desde cuando se realizaron los tests (ej: "2023-01-01")
        test_to_date: Fecha hasta (opcional)
    """
    
    print(f"🎯 MÉTODO FINAL: Obtener todos los datos disponibles y filtrar")
    print(f"📅 Filtrar tests realizados desde: {test_from_date}")
    
    # La API solo permite datos recientes, así que obtenemos todos los disponibles
    # Usar fecha de modificación que permita obtener el máximo de datos
    modified_from_recent = "2024-01-01T00:00:00.000Z"  # Fecha más reciente que puede funcionar
    
    # Obtener datos usando el endpoint que SÍ funciona
    df = get_nordbord_working_endpoint(token, tenant_id, modified_from_recent)
    
    if df.empty:
        print("⚠️ No se obtuvieron datos del endpoint")
        return df
    
    print(f"📊 Total de registros obtenidos: {len(df)}")
    
    # Inspeccionar columnas de fecha disponibles
    date_columns = [col for col in df.columns if 'date' in col.lower() or 'time' in col.lower()]
    print(f"📅 Columnas de fecha disponibles: {date_columns}")
    
    # Buscar la columna de fecha del test
    test_date_col = None
    
    # Prioridad de búsqueda
    priority_columns = ['testDateUtc', 'testDate', 'dateUtc', 'date']
    
    for col_name in priority_columns:
        if col_name in df.columns:
            test_date_col = col_name
            print(f"✅ Usando columna de fecha del test: '{test_date_col}'")
            break
    
    if not test_date_col:
        # Si no encuentra las prioritarias, usar la primera que no sea 'modified'
        non_modified_dates = [col for col in date_columns if 'modified' not in col.lower()]
        if non_modified_dates:
            test_date_col = non_modified_dates[0]
            print(f"📅 Usando primera columna de fecha no-modificada: '{test_date_col}'")
        else:
            print("⚠️ No se encontró columna adecuada para fecha del test")
            print(f"📋 Columnas disponibles: {list(df.columns)}")
            return df
    
    # Convertir columna de fecha a datetime
    try:
        df[test_date_col] = pd.to_datetime(df[test_date_col])
        print(f"✅ Columna '{test_date_col}' convertida a datetime")
    except Exception as e:
        print(f"❌ Error convirtiendo '{test_date_col}' a datetime: {e}")
        return df
    
    # Mostrar rango de fechas disponibles
    min_date = df[test_date_col].min()
    max_date = df[test_date_col].max()
    print(f"📊 Rango de fechas en los datos: {min_date.strftime('%Y-%m-%d')} a {max_date.strftime('%Y-%m-%d')}")
    
    # Aplicar filtro por fecha del test (con timezone UTC)
    test_from_dt = pd.to_datetime(test_from_date, utc=True)
    print(f"🔍 Filtrando tests realizados desde: {test_from_dt.strftime('%Y-%m-%d')}")
    
    # Aplicar filtro
    filtered_df = df[df[test_date_col] >= test_from_dt].copy()
    
    if test_to_date:
        test_to_dt = pd.to_datetime(test_to_date, utc=True)
        print(f"🔍 Filtrando tests realizados hasta: {test_to_dt.strftime('%Y-%m-%d')}")
        filtered_df = filtered_df[filtered_df[test_date_col] <= test_to_dt]
    
    print(f"🎉 Filtrado completado: {len(filtered_df)} registros de {len(df)} totales")
    
    if len(filtered_df) > 0:
        filtered_min = filtered_df[test_date_col].min()
        filtered_max = filtered_df[test_date_col].max()
        print(f"📊 Rango final: {filtered_min.strftime('%Y-%m-%d')} a {filtered_max.strftime('%Y-%m-%d')}")
        
        # Mostrar distribución por año
        filtered_df['year'] = filtered_df[test_date_col].dt.year
        year_counts = filtered_df['year'].value_counts().sort_index()
        print(f"📈 Distribución por año:")
        for year, count in year_counts.items():
            print(f"   {year}: {count} tests")
        filtered_df = filtered_df.drop('year', axis=1)
    else:
        print("⚠️ No se encontraron tests en el rango de fechas especificado")
    
    return filtered_df


def get_nordbord_working_endpoint(token, tenant_id, fecha_desde, profile_id=None):
    """
    Función que usa el endpoint /tests/v2 que sabemos que funciona
    (Optimizada para obtener TODOS los datos disponibles)
    """
    base_url = "https://prd-use-api-externalnordbord.valdperformance.com"
    endpoint = "/tests/v2"
    
    headers = {"Authorization": f"Bearer {token}"}
    
    all_tests = []
    current_modified_from = fecha_desde
    page_count = 0
    
    print(f"📡 Extrayendo datos desde API NordBord (tenant: {tenant_id})")
    print(f"⏰ Usando modifiedFromUtc: {fecha_desde}")
    
    while True:
        page_count += 1
        
        # Log menos frecuente para no saturar
        if page_count == 1 or page_count % 20 == 0:
            print(f"📄 Página {page_count} - {len(all_tests)} registros acumulados")
        
        params = {
            "tenantId": tenant_id,
            "modifiedFromUtc": current_modified_from
        }
        
        if profile_id:
            params["profileId"] = profile_id
        
        try:
            response = requests.get(f"{base_url}{endpoint}", params=params, headers=headers)
            
            if response.status_code == 200:
                datos = response.json()
                
                # Determinar estructura de datos
                if isinstance(datos, list):
                    current_batch = datos
                elif isinstance(datos, dict):
                    current_batch = datos.get('tests', datos.get('items', datos.get('data', [datos])))
                else:
                    current_batch = [datos] if datos else []
                
                if not current_batch:
                    print("✅ No hay más datos")
                    break
                
                all_tests.extend(current_batch)
                
                # Lógica de paginación anti-bucle infinito
                last_record = current_batch[-1]
                next_modified_date = None
                
                for field in ['modifiedDateUtc', 'modifiedDate', 'lastModified', 'updatedAt']:
                    if field in last_record:
                        next_modified_date = last_record[field]
                        break
                
                if next_modified_date:
                    if next_modified_date == current_modified_from:
                        # Detectado bucle infinito - agregar milisegundo
                        try:
                            dt = datetime.fromisoformat(next_modified_date.replace('Z', '+00:00'))
                            dt_next = dt + timedelta(milliseconds=1)
                            current_modified_from = dt_next.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z'
                        except Exception as e:
                            print(f"❌ Error procesando fecha: {e}")
                            break
                    else:
                        current_modified_from = next_modified_date
                else:
                    print("⚠️ No se encontró campo de paginación")
                    break
                
                # Límite de seguridad
                if page_count > 3000:
                    print("⚠️ Alcanzado límite máximo de páginas (3000)")
                    break
                
            elif response.status_code == 204:
                print("✅ No hay más registros disponibles")
                break
                
            else:
                print(f"❌ Error HTTP {response.status_code}: {response.text}")
                break
                
        except Exception as e:
            print(f"❌ Error en solicitud: {str(e)}")
            break
    
    # Procesar resultados
    if all_tests:
        df = pd.DataFrame(all_tests)
        
        # Convertir todas las columnas de fecha
        date_columns = [col for col in df.columns if 'date' in col.lower() or 'time' in col.lower()]
        for col in date_columns:
            try:
                df[col] = pd.to_datetime(df[col])
            except:
                pass  # Mantener original si no se puede convertir
        
        df['tenant_id'] = tenant_id
        
        print(f"🎉 Extracción completada: {len(df)} registros en {page_count} páginas")
        return df
    else:
        print("⚠️ No se obtuvieron datos")
        return pd.DataFrame()


def debug_nordbord_structure(token, tenant_id):
    """
    Función de debug para inspeccionar la estructura de datos
    """
    print("🔍 DEBUG: Inspeccionando estructura de datos NordBord")
    
    # Obtener una muestra pequeña con fecha reciente
    recent_date = "2025-01-01T00:00:00.000Z"
    df_sample = get_nordbord_working_endpoint(token, tenant_id, recent_date)
    
    if not df_sample.empty:
        print(f"\n📊 Muestra obtenida: {len(df_sample)} registros")
        print(f"📋 Todas las columnas ({len(df_sample.columns)}):")
        
        for i, col in enumerate(df_sample.columns, 1):
            print(f"  {i:2d}. {col}")
        
        # Mostrar columnas de fecha con ejemplos
        date_cols = [col for col in df_sample.columns if 'date' in col.lower() or 'time' in col.lower()]
        print(f"\n📅 Columnas de fecha ({len(date_cols)}):")
        
        for col in date_cols:
            # Obtener valores únicos (sin NaN)
            unique_values = df_sample[col].dropna().unique()[:3]
            print(f"  - {col}: {list(unique_values)}")
        
        # Mostrar una fila completa como ejemplo
        print(f"\n📋 Ejemplo de un registro:")
        first_row = df_sample.iloc[0]
        for col, value in first_row.items():
            if len(str(value)) < 100:  # Solo valores cortos
                print(f"  {col}: {value}")
        
        return df_sample
    else:
        print("❌ No se pudo obtener muestra para debug")
        return pd.DataFrame()


# FUNCIÓN PRINCIPAL PARA USAR EN TU CÓDIGO
def get_nordbord_final(token, tenant_id, test_from_date="2023-01-01"):
    """
    Función principal para obtener datos NordBord filtrados por fecha del test
    
    Esta es la función que debes usar en lugar de get_nordbord_complete
    """
    print(f"🚀 Iniciando extracción NordBord filtrada por fecha del test")
    
    try:
        df = get_nordbord_complete_and_filter(token, tenant_id, test_from_date)
        
        if df.empty:
            print("🔍 Ejecutando debug para inspeccionar estructura...")
            debug_nordbord_structure(token, tenant_id)
        
        return df
        
    except Exception as e:
        print(f"❌ Error en extracción: {e}")
        print("🔍 Ejecutando debug...")
        debug_nordbord_structure(token, tenant_id)
        return pd.DataFrame()


# PARA USAR EN TU EXTRACTOR.PY:
# Reemplaza:
#   df_nordbord = get_nordbord_complete(token, tenant_id, FECHA_DESDE)
# Por:
#   df_nordbord = get_nordbord_final(token, tenant_id, "2023-01-01")

In [24]:
df_nordbord_fixed = get_nordbord_final(token, tenant_id, "2023-01-01")
print(f"📊 Registros obtenidos: {len(df_nordbord_fixed)}")

🚀 Iniciando extracción NordBord filtrada por fecha del test
🎯 MÉTODO FINAL: Obtener todos los datos disponibles y filtrar
📅 Filtrar tests realizados desde: 2023-01-01
📡 Extrayendo datos desde API NordBord (tenant: f1185650-fb79-44a0-8b4b-b2bf82d28c83)
⏰ Usando modifiedFromUtc: 2024-01-01T00:00:00.000Z
📄 Página 1 - 0 registros acumulados
✅ No hay más registros disponibles
🎉 Extracción completada: 349 registros en 9 páginas
📊 Total de registros obtenidos: 349
📅 Columnas de fecha disponibles: ['modifiedDateUtc', 'testDateUtc']
✅ Usando columna de fecha del test: 'testDateUtc'
✅ Columna 'testDateUtc' convertida a datetime
📊 Rango de fechas en los datos: 2023-05-15 a 2025-05-13
🔍 Filtrando tests realizados desde: 2023-01-01
🎉 Filtrado completado: 349 registros de 349 totales
📊 Rango final: 2023-05-15 a 2025-05-13
📈 Distribución por año:
   2023: 4 tests
   2024: 226 tests
   2025: 119 tests
📊 Registros obtenidos: 349


In [26]:
len(df_nordbord_fixed)

349

In [27]:
# Para obtener solo tests de 2024 en adelante
df_2024 = get_nordbord_final(token, tenant_id, "2024-01-01")

# Para obtener solo tests de 2025
df_2025 = get_nordbord_final(token, tenant_id, "2025-01-01")

🚀 Iniciando extracción NordBord filtrada por fecha del test
🎯 MÉTODO FINAL: Obtener todos los datos disponibles y filtrar
📅 Filtrar tests realizados desde: 2024-01-01
📡 Extrayendo datos desde API NordBord (tenant: f1185650-fb79-44a0-8b4b-b2bf82d28c83)
⏰ Usando modifiedFromUtc: 2024-01-01T00:00:00.000Z
📄 Página 1 - 0 registros acumulados
✅ No hay más registros disponibles
🎉 Extracción completada: 349 registros en 9 páginas
📊 Total de registros obtenidos: 349
📅 Columnas de fecha disponibles: ['modifiedDateUtc', 'testDateUtc']
✅ Usando columna de fecha del test: 'testDateUtc'
✅ Columna 'testDateUtc' convertida a datetime
📊 Rango de fechas en los datos: 2023-05-15 a 2025-05-13
🔍 Filtrando tests realizados desde: 2024-01-01
🎉 Filtrado completado: 345 registros de 349 totales
📊 Rango final: 2024-01-09 a 2025-05-13
📈 Distribución por año:
   2024: 226 tests
   2025: 119 tests
🚀 Iniciando extracción NordBord filtrada por fecha del test
🎯 MÉTODO FINAL: Obtener todos los datos disponibles y filt

In [119]:
# Obtener datos de ForceFrame
print("\n📊 Obteniendo datos de Nordbord...")
df_nordbord3 = get_nordbord_complete_fixed(token, tenant_id, '2023-01-01T00:00:00.000Z')


📊 Obteniendo datos de Nordbord...
🔄 NUEVA VERSIÓN: Filtrando por fecha del test (no modificación)
📅 Tests realizados desde: 2023-01-01T00:00:00.000Z

🔄 Método 1: Usando períodos de 6 meses
🗓️ Dividiendo extracción desde 2023-01-01T00:00:00.000Z hasta 2025-08-19T23:59:59.999Z en períodos de 6 meses

📅 Período 1: 2023-01-01T00:00:00.000Z a 2023-07-01T23:59:59.999Z
🔄 Obteniendo tests NordBord realizados entre 2023-01-01T00:00:00.000Z y 2023-07-01T23:59:59.999Z
📡 Usando endpoint /tests con paginación tradicional
📄 Procesando página 1
❌ Error 400: {"errors":{"TestFromUtc":["'TestFromUtc' must be greater than or equal to '1/1/2023 11:59:59 pm'."]},"type":"https://tools.ietf.org/html/rfc9110#section-15.5.1","title":"One or more validation errors occurred.","status":400,"traceId":"00-7a1fb17fbfb822328ee9b5e5023d954d-a51ec6de21369a0a-00"}
⚠️ No se obtuvieron datos

📅 Período 2: 2023-07-01T00:00:00.000Z a 2024-01-01T23:59:59.999Z
🔄 Obteniendo tests NordBord realizados entre 2023-07-01T00:00:00.

In [ ]:
# exportar la notebook a html 
